In [1]:
import torch

from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 128
NUM_EPOCHS = 30

# Architecture
NUM_CLASSES = 10

# Other
DEVICE = "cuda:0"
GRAYSCALE = False

In [5]:
train_dataset = datasets.CIFAR10(root='data', 
                                 train=True, 
                                 transform=transforms.ToTensor(),
                                 download=True)

test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

valid_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False)

Files already downloaded and verified


In [6]:
device = torch.device(DEVICE)
torch.manual_seed(RANDOM_SEED)


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict



def _bn_function_factory(norm, relu, conv):
    def bn_function(*inputs):
        concated_features = torch.cat(inputs, 1)
        bottleneck_output = conv(relu(norm(concated_features)))
        return bottleneck_output

    return bn_function


class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate, memory_efficient=False):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm2d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size *
                                           growth_rate, kernel_size=1, stride=1,
                                           bias=False)),
        self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                                           kernel_size=3, stride=1, padding=1,
                                           bias=False)),
        self.drop_rate = drop_rate
        self.memory_efficient = memory_efficient

    def forward(self, *prev_features):
        bn_function = _bn_function_factory(self.norm1, self.relu1, self.conv1)
        if self.memory_efficient and any(prev_feature.requires_grad for prev_feature in prev_features):
            bottleneck_output = cp.checkpoint(bn_function, *prev_features)
        else:
            bottleneck_output = bn_function(*prev_features)
        new_features = self.conv2(self.relu2(self.norm2(bottleneck_output)))
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate,
                                     training=self.training)
        return new_features


class _DenseBlock(nn.Module):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate, memory_efficient=False):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(
                num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient,
            )
            self.add_module('denselayer%d' % (i + 1), layer)

    def forward(self, init_features):
        features = [init_features]
        for name, layer in self.named_children():
            new_features = layer(*features)
            features.append(new_features)
        return torch.cat(features, 1)


class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))


class DenseNet121(nn.Module):

    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_featuremaps=64, bn_size=4, drop_rate=0, num_classes=10, memory_efficient=False,
                 grayscale=False):

        super(DenseNet121, self).__init__()

        if grayscale:
            in_channels=1
        else:
            in_channels=3
        
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(in_channels=in_channels, out_channels=num_init_featuremaps,
                                kernel_size=7, stride=2,
                                padding=3, bias=False)), # bias is redundant when using batchnorm
            ('norm0', nn.BatchNorm2d(num_features=num_init_featuremaps)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
        ]))

        num_features = num_init_featuremaps
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient
            )
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features,
                                    num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        self.features.add_module('norm5', nn.BatchNorm2d(num_features))

        self.classifier = nn.Linear(num_features, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = torch.flatten(out, 1)
        logits = self.classifier(out)
        probas = F.softmax(logits, dim=1)
        return logits, probas

In [8]:
torch.manual_seed(RANDOM_SEED)

model = DenseNet121(num_classes=NUM_CLASSES, grayscale=GRAYSCALE)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

In [9]:
import keyboard
import time

def simulate_key_press(key):
    keyboard.press(key)
    time.sleep(0.1)  # Halte die Taste für 0.1 Sekunden gedrückt
    keyboard.release(key)


In [10]:
def compute_acc(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    model.eval()
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        assert predicted_labels.size() == targets.size()
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [12]:
time.sleep(1)
simulate_key_press('p')
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
            
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        optimizer.step()

    model.eval()
    
    print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d}\n')
        
    elapsed = (time.time() - start_time)
    
total_training_time = (time.time() - start_time)

total_accuracy = compute_acc(model, valid_loader, device=DEVICE)
print(f'Total Training Time: {total_training_time:.2f} sec')
print(f'Total Training Time: {total_accuracy:.2f} %')
time.sleep(1)
simulate_key_press('p')

Epoch: 001/001, Batch: 001, Batch Time: 192.04 ms
Epoch: 001/001, Batch: 002, Batch Time: 73.02 ms
Epoch: 001/001, Batch: 003, Batch Time: 73.02 ms
Epoch: 001/001, Batch: 004, Batch Time: 76.02 ms
Epoch: 001/001, Batch: 005, Batch Time: 73.02 ms
Epoch: 001/001, Batch: 006, Batch Time: 72.02 ms
Epoch: 001/001, Batch: 007, Batch Time: 75.02 ms
Epoch: 001/001, Batch: 008, Batch Time: 72.02 ms
Epoch: 001/001, Batch: 009, Batch Time: 72.02 ms
Epoch: 001/001, Batch: 010, Batch Time: 76.02 ms
Epoch: 001/001, Batch: 011, Batch Time: 72.01 ms
Epoch: 001/001, Batch: 012, Batch Time: 72.02 ms
Epoch: 001/001, Batch: 013, Batch Time: 73.02 ms
Epoch: 001/001, Batch: 014, Batch Time: 71.02 ms
Epoch: 001/001, Batch: 015, Batch Time: 72.02 ms
Epoch: 001/001, Batch: 016, Batch Time: 74.02 ms
Epoch: 001/001, Batch: 017, Batch Time: 72.02 ms
Epoch: 001/001, Batch: 018, Batch Time: 72.02 ms
Epoch: 001/001, Batch: 019, Batch Time: 74.02 ms
Epoch: 001/001, Batch: 020, Batch Time: 72.02 ms
Epoch: 001/001, Bat

In [44]:
import csv
import os

#Zum loggen der Trainingslaufzeit und accuracy  des Models

def add_row_to_csv(filename, datensatz_name, frequency, voltage, runtime, accuracy):
    # Daten für die neue Zeile
    new_row = {'Datensatz': datensatz_name, 'Frequenz': frequency, 'voltage': voltage, 'Laufzeit': runtime, 'Accuracy': accuracy}
    
    # Überprüfe, ob die Datei existiert
    file_exists = os.path.isfile(filename)
    
    # Öffne die CSV-Datei im Modus 'a' (append), um Werte hinzuzufügen
    with open(filename, 'a', newline='') as csvfile:
        # Erstelle einen CSV-Writer, wenn die Datei neu erstellt wird
        writer = csv.DictWriter(csvfile, fieldnames=['Datensatz', 'Frequenz', 'voltage', 'Laufzeit', 'Accuracy'])
        
        # Schreibe die Kopfzeile, falls die Datei neu erstellt wurde
        if not file_exists:
            writer.writeheader()
        
        # Schreibe die neue Zeile in die CSV-Datei
        writer.writerow(new_row)

# Beispielaufruf der Funktion
filename = 'densenet121_cifar_spannung.csv'
datensatz_name = 'cifar'
frequency = '3600'
voltage = '1.1'
runtime = round(total_training_time, 2)
print(total_accuracy)
accuracy = round(total_accuracy.item(), 2)

add_row_to_csv(filename, datensatz_name, frequency, voltage, runtime, accuracy)


tensor(77.1000, device='cuda:0')
